<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [1]:
import numpy as np #se importa numpy como np

In [2]:
class RedNeuronal:
    # num_entrada: dimension de los vectores de entrada, en iris: 4
    # tam_capas: lista de cantidad de neuronas por cada capa interna, en iris: [32, 16]
    # num_salida: dimension de los vectores de salida, en iris: 3
    def __init__(self, num_entrada, tam_capas, num_salida):
        self.num_entrada = num_entrada
        self.tam_capas = tam_capas
        self.num_salida = num_salida
        self.capas = []
        # iniciar todas las capas inicial - ocultas - final
        self.capas.append(CapaNeuronal(num_entrada))
        for i in range(len(tam_capas)):
            self.capas.append(CapaNeuronal(tam_capas[i], self.capas[i].tam_capa))
        self.capas.append(CapaNeuronal(num_salida, self.capas[len(self.capas)-1].tam_capa))
        self.cant_capas = len(self.capas)
        
    def crossentropy(self, prediccion, esperado, epsilon=1e-12): #El epsilon evita log(0)
        prediccion = np.clip(prediccion, epsilon, 1. - epsilon)
        N = prediccion.shape[0]
        ce = -np.sum(np.sum(esperado*np.log(prediccion+1e-9)))/N
        return ce
    
    def crossentropy_derivative(self, prediccion, esperado):
        return -(esperado-1/prediccion+(1+esperado)*(1/(1-prediccion)))    
    
    def softmax_derivative(self, x):
        res = []
        for i in x:
            res.append(np.exp(i)*(np.sum(np.exp(x), axis=0)-np.exp(i))/(np.sum(np.exp(x), axis=0)**2))
        return np.array(res)
    
    def train(self, entrada, output, ciclos, tasa):
        # repetir para todas 
        last = self.cant_capas-1 # indice capa de salida
        for ciclo in range(ciclos):
            iteracion = 0
            errores_epoch = 0 # Error general del ciclo. Almacena un escalar por cada entrada
            errores_derivadas = np.zeros(self.num_salida) # Error de crossentropy()' almanacena por neurona final el error
            error_softmax = np.zeros(16) # Error entre capa de salida y ultima capa oculta
            error_capa2 = np.zeros(16) # ALmacena activacion de cada neurona de la ultima capa oculta
            # Comienza 1 epoch
            for i in range(len(entrada)):
                i+=1
                self.capas[0].set_capa(entrada[i]) #Poblar capa inicial
                
                # Feed forward
                for j in range(1, len(self.capas)-1):
                    self.capas[j].feed_forward(self.capas[j-1])
                self.capas[last].softmax_forward(self.capas[last-1])
                
                # evaluar error
                errores_epoch += self.crossentropy(self.capas[last].get_activaciones(), output[i])                    
                errores_derivadas += self.crossentropy_derivative(self.capas[last].get_activaciones(), output[i])
                error_softmax += self.softmax_derivative(self.capas[last-1].get_activaciones())
                error_capa2 += self.capas[last-1].get_activaciones()
                #self.capas[last].mostrar_capa()
            #Error final de cada epoch    
            error_final = errores_epoch/iteracion
            ce_derivative = errores_derivadas/iteracion #promedio de errores por neurona de salida
            softmax_derivative = error_softmax/iteracion #promedio errores por pesos de capa de salida segun capa anterior
            error_capa2 = error_capa2/iteracion #promedio de activaciones de la capa 2
            error_capa2 = np.transpose(np.tile(error_capa2, (3,1)))
            print("Error del epoch: "+str(error_final))
            error = np.multiply(np.outer(softmax_derivative,ce_derivative),error_capa2)
            #pesos actualizados de la ULTIMA CAPA (SALIDA)
            self.capas[last].actualizar_pesos(error, tasa)
            # actualiacion de pesos capas ocultas
            
                # Back propagation (actualizar pesos de las neuronas)
            



In [3]:
#Felipe Code

def set_peso(self,tasa,derivada): #neurona.pesos
    return neurona.pesos - tasa*derivada




In [4]:
class CapaNeuronal:
    def __init__(self, tam_capa, tam_anterior = 0):
        self.tam_capa = tam_capa
        self.bias = 1
        self.neuronas = [Neurona(tam_anterior) for i in range(tam_capa)] # lista de neuronas de la capa
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
        
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)
    
    def get_activaciones(self):
        return np.array([neurona.activacion for neurona in self.neuronas])
    
    def feed_forward(self, capa_anterior):
        for neurona in self.neuronas:
            activaciones = capa_anterior.get_activaciones()
            pesos = neurona.pesos
            result = self.sigmoid(np.dot(pesos, activaciones) + self.bias)          
            neurona.set_activacion(result)
    
    def softmax_forward(self, capa_anterior):
        pre_softmax = []
        for neurona in self.neuronas:
            activaciones = capa_anterior.get_activaciones()
            pesos = neurona.pesos
            pre_softmax.append(np.dot(pesos, activaciones) + self.bias)
        self.set_capa(self.softmax(pre_softmax))
    
    def actualizar_pesos(self, matriz, rate):
        pesos = np.transpose(np.array([n.pesos for n in self.neuronas]))
        pesos += rate*matriz #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].pesos = pesos[:,i]
    #Funcion para poblar datos de entrada a la red.
    def set_capa(self, valores):
        for i in range(len(self.neuronas)):
            self.neuronas[i].set_activacion(valores[i])
            
    def mostrar_capa(self):
        for neurona in self.neuronas:
            print(neurona, end="\t")
        print("\n-----------------------------")

        
class Neurona:
    def __init__(self, tam_capa_anterior):
        self.pesos = np.random.rand(tam_capa_anterior) # genera una lista de numeros aleatorios de tamaño "tam_capa_anterior"

    def set_activacion(self, valor):
        self.activacion = valor
        
    def __str__(self):
        return "Activación: "+str(self.activacion)

### ¡Es hora de probar!

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import keras
x_train,y_train = load_iris(return_X_y=True)
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
y_onehot = keras.utils.to_categorical(y_train)
#transform target to one hot vector
nn = RedNeuronal(4, [32, 16], 3)
nn.train(x_train, y_onehot, 1, 0.1)

## 1. Referencias


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Pregunta 2

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer=SGD(lr=0.1),loss="mse", metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_onehot, epochs=100, batch_size=1, verbose=1)